In [48]:
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd

from tsfresh.feature_extraction import feature_calculators # time series feature exctraction
import librosa # music/aucustic signal manipulation, used here to mfcc feature exctraction
import pywt # wavelet transform library


from joblib import Parallel, delayed
import scipy as sp
import itertools
import gc

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from numpy import random
import lightgbm as lgb

from hyperopt import hp, tpe, Trials, STATUS_OK
from hyperopt.fmin import fmin
from hyperopt.pyll.stochastic import sample

from fastai.datasets import Config

In [17]:
base_path = Config.data_path()
data_path = base_path/'LANL_Earthquake_Prediction'
competition_name = 'LANL-Earthquake-Prediction'
(data_path/'temp').mkdir(exist_ok=True)
data_path

PosixPath('/home/krzysiek/.fastai/data/LANL_Earthquake_Prediction')

In [18]:
data_path.ls()

[PosixPath('/home/krzysiek/.fastai/data/LANL_Earthquake_Prediction/submissions'),
 PosixPath('/home/krzysiek/.fastai/data/LANL_Earthquake_Prediction/train'),
 PosixPath('/home/krzysiek/.fastai/data/LANL_Earthquake_Prediction/test'),
 PosixPath('/home/krzysiek/.fastai/data/LANL_Earthquake_Prediction/test_processed_df'),
 PosixPath('/home/krzysiek/.fastai/data/LANL_Earthquake_Prediction/train.csv'),
 PosixPath('/home/krzysiek/.fastai/data/LANL_Earthquake_Prediction/test.zip'),
 PosixPath('/home/krzysiek/.fastai/data/LANL_Earthquake_Prediction/sample_submission.csv'),
 PosixPath('/home/krzysiek/.fastai/data/LANL_Earthquake_Prediction/temp')]

In [8]:
%%time
# Load training samples
raw = pd.read_csv(
    data_path/'train.csv', 
    dtype={
        'acoustic_data': np.int16, 
        'time_to_failure': np.float64
    })


CPU times: user 1min 50s, sys: 32.7 s, total: 2min 23s
Wall time: 2min 42s


In [20]:
np.random.seed(1337)
noise = np.random.normal(0, 0.5, 150_000)

def denoise_signal_simple(x, wavelet='db4', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    #univeral threshold
    uthresh = 10
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    # Reconstruct the signal using the thresholded coefficients
    return pywt.waverec(coeff, wavelet, mode='per')

def feature_gen(z):
    X = pd.DataFrame(index=[0], dtype=np.float64)
    
    z = z + noise
    z = z - np.median(z)

    den_sample_simple = denoise_signal_simple(z)
    mfcc = librosa.feature.mfcc(z)
    mfcc_mean = mfcc.mean(axis=1)
    percentile_roll50_std_20 = np.percentile(pd.Series(z).rolling(50).std().dropna().values, 20)
    
    X['var_num_peaks_2_denoise_simple'] = feature_calculators.number_peaks(den_sample_simple, 2)
    X['var_percentile_roll50_std_20'] = percentile_roll50_std_20
    X['var_mfcc_mean18'] = mfcc_mean[18]
    X['var_mfcc_mean4'] = mfcc_mean[4]
    
    return X

def parse_sample(sample, start):
    delta = feature_gen(sample['acoustic_data'].values)
    delta['start'] = start
    delta['target'] = sample['time_to_failure'].values[-1]
    return delta
    
def sample_train_gen(df, segment_size=150_000, indices_to_calculate=[0]):
    result = Parallel(n_jobs=4, temp_folder=data_path/'temp', max_nbytes=None, backend="multiprocessing")(delayed(parse_sample)(df[int(i) : int(i) + segment_size], int(i)) 
                                                                                                for i in tqdm(indices_to_calculate))
    data = [r.values for r in result]
    data = np.vstack(data)
    X = pd.DataFrame(data, columns=result[0].columns)
    X = X.sort_values("start")
    
    return X

def parse_sample_test(seg_id):
    sample = pd.read_csv(data_path/'test'/f"{seg_id}.csv", dtype={'acoustic_data': np.int32})
    delta = feature_gen(sample['acoustic_data'].values)
    delta['seg_id'] = seg_id
    return delta

def sample_test_gen():
    X = pd.DataFrame()
    submission = pd.read_csv(data_path/'sample_submission.csv', index_col='seg_id')
    result = Parallel(n_jobs=4, temp_folder=data_path/'temp', max_nbytes=None, backend="multiprocessing")(delayed(parse_sample_test)(seg_id) for seg_id in tqdm(submission.index))
    data = [r.values for r in result]
    data = np.vstack(data)
    X = pd.DataFrame(data, columns=result[0].columns)
    
    return X

In [21]:
indices_to_calculate = raw.index.values[::150_000][:-1]

train = sample_train_gen(raw, indices_to_calculate=indices_to_calculate)
gc.collect()
test = sample_test_gen()

In [22]:
etq_meta = [
    {"start":0,         "end":5656574},
    {"start":5656574,   "end":50085878},
    {"start":50085878,  "end":104677356},
    {"start":104677356, "end":138772453},
    {"start":138772453, "end":187641820},
    {"start":187641820, "end":218652630},
    {"start":218652630, "end":245829585},
    {"start":245829585, "end":307838917},
    {"start":307838917, "end":338276287},
    {"start":338276287, "end":375377848},
    {"start":375377848, "end":419368880},
    {"start":419368880, "end":461811623},
    {"start":461811623, "end":495800225},
    {"start":495800225, "end":528777115},
    {"start":528777115, "end":585568144},
    {"start":585568144, "end":621985673},
    {"start":621985673, "end":629145480},
]

for i, etq in enumerate(etq_meta):
    train.loc[(train['start'] + 150_000 >= etq["start"]) & (train['start'] <= etq["end"] - 150_000), "eq"] = i

train_sample = train[train["eq"].isin([2, 7, 0, 4, 11, 13, 9, 1, 14, 10])]

In [25]:
print(f"Mean:   {train_sample['target'].mean():.4}")
print(f"Median: {train_sample['target'].median():.4}")

Mean:   6.258
Median: 6.031


In [28]:
random.seed(1234)

features = ['var_num_peaks_2_denoise_simple','var_percentile_roll50_std_20','var_mfcc_mean4',  'var_mfcc_mean18']
target = train_sample["target"].values

train_X = train_sample[features].values
test_X = test[features].values

submission = pd.read_csv(data_path/'sample_submission.csv', index_col='seg_id')
oof = np.zeros(len(train_X))
prediction = np.zeros(len(submission))

n_fold = 3

kf = KFold(n_splits=n_fold, shuffle=True, random_state=1337)
kf = list(kf.split(np.arange(len(train_sample))))

for fold_n, (train_index, valid_index) in enumerate(kf):
    print('Fold', fold_n)

    trn_data = lgb.Dataset(train_X[train_index], label=target[train_index])
    val_data = lgb.Dataset(train_X[valid_index], label=target[valid_index])
    
    params = {'num_leaves': 4,
      'min_data_in_leaf': 5,
      'objective':'fair',
      'max_depth': -1,
      'learning_rate': 0.02,
      "boosting": "gbdt",
      'boost_from_average': True,
      "feature_fraction": 0.9,
      "bagging_freq": 1,
      "bagging_fraction": 0.5,
      "bagging_seed": 0,
      "metric": 'mae',
      "verbosity": -1,
      'max_bin': 500,
      'reg_alpha': 0,
      'reg_lambda': 0,
      'seed': 0,
      'n_jobs': 1
      }

    clf = lgb.train(params, trn_data, 1000000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 1000)

    oof[valid_index] += clf.predict(train_X[valid_index], num_iteration=clf.best_iteration)
    prediction += clf.predict(test_X, num_iteration=clf.best_iteration)

prediction /= n_fold

print('\nMAE: ', mean_absolute_error(target, oof))

Fold 0
Training until validation scores don't improve for 1000 rounds.
[1000]	training's l1: 1.59127	valid_1's l1: 1.97283
Early stopping, best iteration is:
[587]	training's l1: 1.66628	valid_1's l1: 1.95632
Fold 1
Training until validation scores don't improve for 1000 rounds.
[1000]	training's l1: 1.63939	valid_1's l1: 1.86458
Early stopping, best iteration is:
[147]	training's l1: 1.88729	valid_1's l1: 1.83689
Fold 2
Training until validation scores don't improve for 1000 rounds.
[1000]	training's l1: 1.62559	valid_1's l1: 1.92958
Early stopping, best iteration is:
[391]	training's l1: 1.74828	valid_1's l1: 1.90434

MAE:  1.8992052473248857


In [44]:
submission['time_to_failure'] = prediction 
submission_file_name = "winning_solution"

submission_path = data_path/'submissions'
submission_path.mkdir(exist_ok=True)
submission_file = submission_path/f"{submission_file_name}.csv"

submission.to_csv(submission_file, line_terminator=os.linesep)
print(submission.head())

            time_to_failure
seg_id                     
seg_00030f         4.188369
seg_0012b5         5.778676
seg_00184e         7.223985
seg_003339        10.427658
seg_0042cc         7.773074


In [45]:
# Only 2 submission allowed per day!
!kaggle competitions submit {competition_name} -f {submission_file} -m "Winning solution"

100%|██████████████████████████████████████| 75.0k/75.0k [00:02<00:00, 27.0kB/s]
Successfully submitted to LANL Earthquake Prediction